In [3]:
import pandas as pd
import geopandas as gpd
import pyarrow
import matplotlib
import matplotlib.pyplot as plt
import geojson
import json
import jenkspy
import numpy as np
from earthpy import clip as cl
from altair import datum
import weightedcalcs as wc
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
pd.options.display.float_format = '{:,.2f}'.format
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
# https://data.ca.gov/dataset/alternative-fuel-station-locations
src = pd.read_csv('https://data.ca.gov/dataset/\
d2136de9-4b98-49f5-b076-022a046cd894/resource/843e18ca-7a96-4485-a9fe-1ecdd9bbb9b5/download/\
alternative-fuel-station-locations.csv', low_memory=False)

In [6]:
stations = pd.DataFrame(src)

In [7]:
stations.columns = stations.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [10]:
stations.head()

fuel_type_code         station_name    street_address  \
0            LPG  City of San Antonio   7000 Culebra Rd   
1            LPG       City of Haltom   2200 Higgins Ln   
2           ELEC              Brink's   555 Dividend Dr   
3            LPG      Scott Petroleum  3225 E Roane Ave   
4             BD        Georgia Power    5195 Minola Dr   

  intersection_directions  plus4 station_phone status_code expected_date  \
0                     NaN    nan           NaN           E           NaN   
1                     NaN    nan           NaN           E           NaN   
2                     NaN    nan           NaN           E           NaN   
3                     NaN    nan  662-258-2681           E           NaN   
4                     NaN    nan           NaN           E           NaN   

  groups_with_access_code access_days_time     cards_accepted bd_blends  \
0                 Private              NaN                NaN       NaN   
1                 Private              NaN                NaN       NaN   
2                 Private              NaN                NaN       NaN   
3                  Public      8am-5pm M-F  Checks Cash D M V       NaN   
4                 Private              NaN                NaN       B20   

  ng_fill_type_code ng_psi  ev_level1_evse_num  ev_level2_evse_num  \
0               NaN    NaN                 nan                 nan   
1               NaN    NaN                 nan                 nan   
2               NaN    NaN                 nan                1.00   
3               NaN    NaN                 nan                 nan   
4               NaN    NaN                 nan                 nan   

   ev_dc_fast_count ev_other_info ev_network ev_network_web  \
0               nan           NaN        NaN            NaN   
1               nan           NaN        NaN            NaN   
2               nan           NaN        NaN            NaN   
3               nan           NaN        NaN            NaN   
4               nan           NaN        NaN            NaN   

  date_last_confirmed     id               updated_at owner_type_code  \
0          08/04/2015  49626  2015-08-04 12:47:04 UTC               P   
1          08/04/2015  49621  2015-08-04 12:42:07 UTC               P   
2          03/04/2015  46373  2015-03-04 14:07:45 UTC               P   
3          06/02/2015  50548  2015-06-02 17:56:35 UTC               P   
4          10/06/2015  27799  2015-10-06 21:52:46 UTC               T   

   federal_agency_id federal_agency_name   open_date hydrogen_status_link  \
0                nan                 NaN         NaN                  NaN   
1                nan                 NaN         NaN                  NaN   
2                nan                 NaN  08/13/2011                  NaN   
3                nan                 NaN         NaN                  NaN   
4                nan                 NaN  09/15/2005                  NaN   

  ng_vehicle_class lpg_primary e85_blender_pump ev_connector_types  \
0              NaN       False              NaN                NaN   
1              NaN       False              NaN                NaN   
2              NaN         NaN              NaN              J1772   
3              NaN       False              NaN                NaN   
4              NaN         NaN              NaN                NaN   

                                  address    latitude    longitude  
0  7000 Culebra Rd\nSan Antonio, TX 78238    29.45051     -98.6265  
1  2200 Higgins Ln\nHaltom City, TX 76111  32.7919569   -97.284843  
2      555 Dividend Dr\nCoppell, TX 75019   32.934559   -97.006112  
3      3225 E Roane Ave\nEupora, MS 39744  33.5405885  -89.2600446  
4      5195 Minola Dr\nLithonia, GA 30038  33.7002004  -84.1781197